# Exact cover

Exact cover algorithm implemented by QUBO in quantum annealer

In [1]:
import numpy as np
import time
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler
import dwave.inspector

## Define sets

In [2]:
U = [0,1,2,3,4,5,6,7,8,9]
V = [[0,1,2], [2,3], [6,7,8,9], [7,9], [4,5,6], [4,5], [9]]

def result_info(sampleset):
    r = sampleset.first.sample
    r = [V[k] for k, v in r.items() if v==1]
    print('Edges: ' + str(r))

## Create qubo

Constraints
- total number of elements in subset should be (len(U))
- each element only in one subset

In [3]:
Q = np.zeros((len(V),len(V)))

# Total elements constraint

max_count = 0

for i in range(len(V)):
    Q[i][i] =- len(V[i])
    max_count += len(V[i])

# each element only in one subset

for a in U:
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            if a in V[i] and a in V[j]:
                Q[i][j] = max_count
print(Q)

[[-3. 17.  0.  0.  0.  0.  0.]
 [ 0. -2.  0.  0.  0.  0.  0.]
 [ 0.  0. -4. 17. 17.  0. 17.]
 [ 0.  0.  0. -2.  0.  0. 17.]
 [ 0.  0.  0.  0. -3. 17.  0.]
 [ 0.  0.  0.  0.  0. -2.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.]]


## Creat BQM from QUBO

In [4]:
bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
#bqm = bqm.relabel_variables(edge_names, inplace=False)

# Local deterministic solver

In [5]:
sampleset = dimod.ExactSolver().sample(bqm)
print(sampleset.truncate(10))

   0  1  2  3  4  5  6 energy num_oc.
0  1  0  1  0  0  1  0   -9.0       1
1  0  1  1  0  0  1  0   -8.0       1
2  1  0  0  1  1  0  0   -8.0       1
3  1  0  0  1  0  1  0   -7.0       1
4  1  0  0  0  1  0  1   -7.0       1
5  1  0  1  0  0  0  0   -7.0       1
6  0  1  0  1  1  0  0   -7.0       1
7  0  1  1  0  0  0  0   -6.0       1
8  0  0  1  0  0  1  0   -6.0       1
9  1  0  0  0  0  1  1   -6.0       1
['BINARY', 10 rows, 10 samples, 7 variables]


In [6]:
result_info(sampleset)

Edges: [[0, 1, 2], [6, 7, 8, 9], [4, 5]]


## Local heuristic classical solver

In [7]:
t1 = time.time()
sampleset2 = SimulatedAnnealingSampler().sample(bqm, num_reads=500)
t2 = time.time()
print('Time used by solver (s): {:.1f}'.format((t2-t1)))
print(sampleset2.aggregate().truncate(10))

Time used by solver (s): 0.1
   0  1  2  3  4  5  6 energy num_oc.
0  1  0  1  0  0  1  0   -9.0     135
1  1  0  0  1  1  0  0   -8.0      53
2  0  1  1  0  0  1  0   -8.0      66
3  1  0  0  0  1  0  1   -7.0      31
4  1  0  1  0  0  0  0   -7.0      31
5  1  0  0  1  0  1  0   -7.0      23
6  0  1  0  1  1  0  0   -7.0      34
7  0  1  0  0  1  0  1   -6.0      21
8  1  0  0  0  0  1  1   -6.0      11
9  0  1  0  1  0  1  0   -6.0      11
['BINARY', 10 rows, 416 samples, 7 variables]


In [8]:
result_info(sampleset2)

Edges: [[0, 1, 2], [6, 7, 8, 9], [4, 5]]


## Quantum solver

In [9]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [10]:
sampleset3 = EmbeddingComposite(machine).sample(bqm, num_reads=500)
print(sampleset3) 

    0  1  2  3  4  5  6 energy num_oc. chain_.
0   1  0  1  0  0  1  0   -9.0     238     0.0
1   0  1  1  0  0  1  0   -8.0      55     0.0
2   1  0  0  1  1  0  0   -8.0      94     0.0
3   1  0  0  1  0  1  0   -7.0      20     0.0
4   0  1  0  1  1  0  0   -7.0      25     0.0
5   1  0  1  0  0  0  0   -7.0      15     0.0
6   1  0  0  0  1  0  1   -7.0      18     0.0
7   0  1  0  1  0  1  0   -6.0       7     0.0
8   1  0  0  0  0  1  1   -6.0       4     0.0
9   0  1  1  0  0  0  0   -6.0       4     0.0
10  0  0  1  0  0  1  0   -6.0       5     0.0
11  0  1  0  0  1  0  1   -6.0       3     0.0
12  1  0  0  0  1  0  0   -6.0       7     0.0
13  1  0  0  1  0  0  0   -5.0       1     0.0
14  1  0  0  0  0  1  0   -5.0       2     0.0
15  0  1  0  1  0  0  0   -4.0       2     0.0
['BINARY', 16 rows, 500 samples, 7 variables]


In [11]:
time = sampleset3.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset3.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(time))
print('Physical qubits used: {}'.format(qubits))

QPU time used (ms): 66.3
Physical qubits used: 7


In [12]:
result_info(sampleset3)

Edges: [[0, 1, 2], [6, 7, 8, 9], [4, 5]]
